# API interaction

## Test with indirect approach

- iapr2022project@gmail.com
- ProjectAPI

In [1]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
import selenium
import time
import pathlib
import json
import re
import numpy as np
from joblib import Parallel, delayed

In [2]:
def setup_browser(headless):
    # Setup browser
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("headless")
    chrome_options.add_experimental_option("prefs", {
        "download.default_directory": str(pathlib.Path().resolve()),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": False
    })
    chrome_options.add_argument("--log-level=3")
    driver = webdriver.Chrome(
        ChromeDriverManager().install(), options=chrome_options)
    params = {'behavior': 'allow',
              'downloadPath': str(pathlib.Path().resolve())}
    driver.execute_cdp_cmd('Page.setDownloadBehavior', params)
    return driver

In [3]:
driver = setup_browser(False)
startup_page = "https://universe.roboflow.com/augmented-startups/playing-cards-ow27d"
driver.get(startup_page)

[WDM] - ====== WebDriver manager ======
[WDM] - Could not get version for google-chrome. Is google-chrome installed?
[WDM] - Get LATEST chromedriver version for None google-chrome
[WDM] - Driver [/home/martin/.wdm/drivers/chromedriver/linux64/102.0.5005.61/chromedriver] found in cache
/tmp/ipykernel_4286/2252163697.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x55661698af33 <unknown>
#1 0x5566166d5118 <unknown>
#2 0x5566166f658d <unknown>
#3 0x5566166f3d5a <unknown>
#4 0x55661672ed3a <unknown>
#5 0x556616728e63 <unknown>
#6 0x5566166fe82a <unknown>
#7 0x5566166ff985 <unknown>
#8 0x5566169cf4cd <unknown>
#9 0x5566169d35ec <unknown>
#10 0x5566169b971e <unknown>
#11 0x5566169d4238 <unknown>
#12 0x5566169ae870 <unknown>
#13 0x5566169f0608 <unknown>
#14 0x5566169f0788 <unknown>
#15 0x556616a0af1d <unknown>
#16 0x7f4a3e869609 <unknown>


In [ ]:
sign_in  = driver.find_element(By.CSS_SELECTOR,"#content > div.navbarContainer > nav > div:nth-child(5) > div > a.signIn.signInButton")
sign_in.click()

In [ ]:
mail = driver.find_element(By.CSS_SELECTOR,"#message > div > div.firebaseui-card-content > form > ul > li:nth-child(1) > button")
mail.click()

In [ ]:
mail_enter = driver.find_element(By.CSS_SELECTOR,"#message > div > form > div.firebaseui-card-content > div > div.firebaseui-textfield.mdl-textfield.mdl-js-textfield.mdl-textfield--floating-label.is-upgraded > input")
mail_enter.send_keys("iapr2022project@gmail.com")

In [ ]:
mail_validation = driver.find_element(By.CSS_SELECTOR,"#message > div > form > div.firebaseui-card-actions > div > button.firebaseui-id-submit.firebaseui-button.mdl-button.mdl-js-button.mdl-button--raised.mdl-button--colored")
mail_validation.click()

In [ ]:
password = driver.find_element(By.CSS_SELECTOR,"#message > div > form > div.firebaseui-card-content > div:nth-child(3) > input")
password.send_keys("ProjectAPI")

In [ ]:
submit_sign_in = driver.find_element(By.CSS_SELECTOR,"#message > div > form > div.firebaseui-card-actions > div.firebaseui-form-actions > button")
submit_sign_in.click()

In [ ]:
a = driver.find_element(By.CSS_SELECTOR,"#fileInput")

In [ ]:
a.send_keys("C:/EPFL/master/MA_2/ImageProcessing/EE-451-labs/final_project/data/symbols/test_image_3.jpg")

In [ ]:
try:
    driver.delete_all_cookies()
    time.sleep(1)
    wrong = driver.find_element(By.CSS_SELECTOR,"#swal2-title")
    time.sleep(1)
    wrong_button = driver.find_element(By.CSS_SELECTOR,"body > div.swal2-container.swal2-center.createDatasetDialog.swal2-backdrop-show > div > div.swal2-actions.rightActions > button.swal2-confirm.actionButton.primary")
    wrong_button.click()
    time.sleep(0.5)
    driver.refresh()
except:
    ...

In [ ]:
annotations =  driver.find_element(By.CSS_SELECTOR,"#inferenceJson")

In [ ]:
s = annotations.text
s = s.replace("\n","")
s = s.replace(" ","")
s = s.replace("\"","")
l = s.split(",")
l = list(map(lambda s: s.split("{")[-1],l))
l = list(map(lambda s: s.split("}")[0],l))

In [ ]:
predictions = {}
for i in range(len(l)//6):
    pred = l[6*i:6*(i+1)]
    card = pred[4]
    if card not in predictions:
        predictions[card] = pred
predictions

In [ ]:
driver.close()

### Automatic Testing

In [4]:
def format_annotations(text):
    s = text
    s = s.replace("\n","")
    s = s.replace(" ","")
    s = s.replace("\"","")
    l = s.split(",")
    l = list(map(lambda s: s.split("{")[-1],l))
    l = list(map(lambda s: s.split("}")[0],l))
    predictions = {}
    for i in range(len(l)//6):
        pred = l[6*i:6*(i+1)]
        card = pred[4]
        if card not in predictions:
            predictions[card] = pred
    return predictions

def retrieve_annotation(image_path,hidden=True):
    driver = setup_browser(hidden)
    driver.delete_all_cookies()
    time.sleep(1)
    startup_page = "https://universe.roboflow.com/augmented-startups/playing-cards-ow27d/1/try"
    driver.get(startup_page)
    ### Interact with browser
    sign_in  = driver.find_element(By.CSS_SELECTOR,"#content > div.navbarContainer > nav > div:nth-child(5) > div > a.signIn.signInButton")
    sign_in.click()
    time.sleep(0.2)
    mail = driver.find_element(By.CSS_SELECTOR,"#message > div > div.firebaseui-card-content > form > ul > li:nth-child(1) > button")
    mail.click()
    time.sleep(0.4)
    mail_enter = driver.find_element(By.CSS_SELECTOR,"#message > div > form > div.firebaseui-card-content > div > div.firebaseui-textfield.mdl-textfield.mdl-js-textfield.mdl-textfield--floating-label.is-upgraded > input")
    mail_enter.send_keys("iapr2022project@gmail.com")
    time.sleep(0.3)
    mail_validation = driver.find_element(By.CSS_SELECTOR,"#message > div > form > div.firebaseui-card-actions > div > button.firebaseui-id-submit.firebaseui-button.mdl-button.mdl-js-button.mdl-button--raised.mdl-button--colored")
    mail_validation.click()
    time.sleep(1.5)
    password = driver.find_element(By.CSS_SELECTOR,"#message > div > form > div.firebaseui-card-content > div:nth-child(3) > input")
    password.send_keys("ProjectAPI")
    time.sleep(0.5)
    submit_sign_in = driver.find_element(By.CSS_SELECTOR,"#message > div > form > div.firebaseui-card-actions > div.firebaseui-form-actions > button")
    submit_sign_in.click()
    time.sleep(1)
    ### Submit image
    anot = driver.find_element(By.CSS_SELECTOR,"#fileInput")
    anot.send_keys(image_path)
    time.sleep(2)
    waiting = True
    while waiting:
        try:
            driver.delete_all_cookies()
            time.sleep(1)
            wrong = driver.find_element(By.CSS_SELECTOR,"#swal2-title")
            wrong_button = driver.find_element(By.CSS_SELECTOR,"body > div.swal2-container.swal2-center.createDatasetDialog.swal2-backdrop-show > div > div.swal2-actions.rightActions > button.swal2-confirm.actionButton.primary")
            wrong_button.click()
            time.sleep(0.5)
            driver.refresh()
            time.sleep(4)
            anot = driver.find_element(By.CSS_SELECTOR,"#fileInput")
            anot.send_keys(image_path)
            time.sleep(2)
        except:
            waiting = False
    annotations =  driver.find_element(By.CSS_SELECTOR,"#inferenceJson")
    ### Format Annotations
    annotations_list = format_annotations(annotations.text)
    driver.close()
    return annotations_list

In [5]:
test_image = "C:/EPFL/master/MA_2/ImageProcessing/EE-451-labs/final_project/data/symbols/test_image_3.jpg"
test_annotations = retrieve_annotation(test_image)

[WDM] - ====== WebDriver manager ======
[WDM] - Could not get version for google-chrome. Is google-chrome installed?
[WDM] - Get LATEST chromedriver version for None google-chrome
[WDM] - Driver [/home/martin/.wdm/drivers/chromedriver/linux64/102.0.5005.61/chromedriver] found in cache
/tmp/ipykernel_4286/2252163697.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x55d7252adf33 <unknown>
#1 0x55d724ff8118 <unknown>
#2 0x55d72501958d <unknown>
#3 0x55d725016d5a <unknown>
#4 0x55d725051d3a <unknown>
#5 0x55d72504be63 <unknown>
#6 0x55d72502182a <unknown>
#7 0x55d725022985 <unknown>
#8 0x55d7252f24cd <unknown>
#9 0x55d7252f65ec <unknown>
#10 0x55d7252dc71e <unknown>
#11 0x55d7252f7238 <unknown>
#12 0x55d7252d1870 <unknown>
#13 0x55d725313608 <unknown>
#14 0x55d725313788 <unknown>
#15 0x55d72532df1d <unknown>
#16 0x7f6702905609 <unknown>


In [6]:
annotations_list = []
base_path = "C:/EPFL/master/MA_2/ImageProcessing/EE-451-labs/final_project/data/symbols/"
images = [base_path+"test_image_2.jpg",base_path+"test_image_3.jpg",base_path+"test_image_4.jpg",base_path+"test_image_5.jpg"]
for im in images:
    annotations_list.append(retrieve_annotation(im))

[WDM] - ====== WebDriver manager ======
[WDM] - Could not get version for google-chrome. Is google-chrome installed?
[WDM] - Get LATEST chromedriver version for None google-chrome
[WDM] - Driver [/home/martin/.wdm/drivers/chromedriver/linux64/102.0.5005.61/chromedriver] found in cache
/tmp/ipykernel_4286/2252163697.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x558c872aef33 <unknown>
#1 0x558c86ff9118 <unknown>
#2 0x558c8701a58d <unknown>
#3 0x558c87017d5a <unknown>
#4 0x558c87052d3a <unknown>
#5 0x558c8704ce63 <unknown>
#6 0x558c8702282a <unknown>
#7 0x558c87023985 <unknown>
#8 0x558c872f34cd <unknown>
#9 0x558c872f75ec <unknown>
#10 0x558c872dd71e <unknown>
#11 0x558c872f8238 <unknown>
#12 0x558c872d2870 <unknown>
#13 0x558c87314608 <unknown>
#14 0x558c87314788 <unknown>
#15 0x558c8732ef1d <unknown>
#16 0x7f57b5231609 <unknown>


### Parallel automatic testing

In [7]:
test = Parallel(n_jobs=-1)(delayed(retrieve_annotation)(im) for im in images)

[WDM] - ====== WebDriver manager ======
[WDM] - ====== WebDriver manager ======
[WDM] - Could not get version for google-chrome. Is google-chrome installed?
[WDM] - Get LATEST chromedriver version for None google-chrome
[WDM] - Could not get version for google-chrome. Is google-chrome installed?
[WDM] - Get LATEST chromedriver version for None google-chrome
[WDM] - ====== WebDriver manager ======
[WDM] - Could not get version for google-chrome. Is google-chrome installed?
[WDM] - Get LATEST chromedriver version for None google-chrome
[WDM] - Driver [/home/martin/.wdm/drivers/chromedriver/linux64/102.0.5005.61/chromedriver] found in cache
/tmp/ipykernel_4286/2252163697.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
[WDM] - Driver [/home/martin/.wdm/drivers/chromedriver/linux64/102.0.5005.61/chromedriver] found in cache
/tmp/ipykernel_4286/2252163697.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service o

[WDM] - ====== WebDriver manager ======
[WDM] - Could not get version for google-chrome. Is google-chrome installed?
[WDM] - Get LATEST chromedriver version for None google-chrome
[WDM] - Driver [/home/martin/.wdm/drivers/chromedriver/linux64/102.0.5005.61/chromedriver] found in cache
/tmp/ipykernel_4286/2252163697.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
[WDM] - Driver [/home/martin/.wdm/drivers/chromedriver/linux64/102.0.5005.61/chromedriver] found in cache
/tmp/ipykernel_4286/2252163697.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x5559b4380f33 <unknown>
#1 0x5559b40cb118 <unknown>
#2 0x5559b40ec58d <unknown>
#3 0x5559b40e9d5a <unknown>
#4 0x5559b4124d3a <unknown>
#5 0x5559b411ee63 <unknown>
#6 0x5559b40f482a <unknown>
#7 0x5559b40f5985 <unknown>
#8 0x5559b43c54cd <unknown>
#9 0x5559b43c95ec <unknown>
#10 0x5559b43af71e <unknown>
#11 0x5559b43ca238 <unknown>
#12 0x5559b43a4870 <unknown>
#13 0x5559b43e6608 <unknown>
#14 0x5559b43e6788 <unknown>
#15 0x5559b4400f1d <unknown>
#16 0x7f0cdec31609 <unknown>


# Test with direct API

In [2]:
!pip install requests-toolbelt


     |████████████████████████████████| 54 kB 1.5 MB/s eta 0:00:01


In [3]:
import io
import cv2 
import requests
from PIL import Image
from requests_toolbelt.multipart.encoder import MultipartEncoder

In [8]:
img = cv2.imread("data/symbols/test_image.jpg")
image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
pilImage = Image.fromarray(image)

buffered = io.BytesIO()
pilImage.save(buffered, quality=100, format="JPEG")

m = MultipartEncoder(fields={'file': ("imageToUpload", buffered.getvalue(), "image/jpeg")})
response = requests.post("https://detect.roboflow.com/your-model/your-model-version?api_key=qjCHEdJ6lphh5qmVKC6i",
                         data=m, headers={'Content-Type': m.content_type})

In [9]:
print(response)

<Response [403]>


In [7]:
print(response.json())

{'message': 'Forbidden'}
